In [11]:
from llm_bases.chatglm6b import ChatGML6B
glm = ChatGML6B()

Loading checkpoint shards: 100%|██████████| 8/8 [00:08<00:00,  1.01s/it]


In [12]:
import sys
try:
    del sys.modules["perm_llm.glm6b.wrapped_layer"]
except:
    pass

In [13]:
from perm_llm.glm6b.wrapped_layer import Attention_GLM_Wrapped, FeedForward_GLM_Wrapped, copy_attention, copy_feedforward
from perm_llm.common.torch_utils import relative_error

In [14]:
layer_id = 20

transformer_layer = glm.condgen.transformer.layers[layer_id].float()
attn_wrapped = Attention_GLM_Wrapped(4096, 32, layer_id)
feedforward_wrapped = FeedForward_GLM_Wrapped(4096, 32, layer_id)
copy_attention(transformer_layer, attn_wrapped)
copy_feedforward(transformer_layer, None, feedforward_wrapped)

In [15]:
import torch
from perm_llm.glm6b.utils import generate_attention_mask, generate_position_ids

In [16]:
xs = torch.rand([10, 1, 4096]).cuda()
transformer_layer.cuda()
attn_wrapped.cuda()
feedforward_wrapped.cuda()
position_ids = generate_position_ids(10, 10).cuda()
attention_mask = generate_attention_mask(10, 10).cuda()

In [17]:
output_raw = transformer_layer(xs, position_ids, attention_mask, layer_id)[0]

In [18]:
# Compute the output by wrapped layers
xs_normalized = transformer_layer.input_layernorm(xs)
attn_out_raw = attn_wrapped(xs_normalized, position_ids)
attn_out = attn_out_raw + (2 * 28) ** 0.5 * xs_normalized
output_wrapped = feedforward_wrapped(attn_out)


In [19]:
print(f"Error of transformer layer protocol: {relative_error(output_wrapped, output_raw):.5f}")

Error of transformer layer protocol: 0.00012
